In [0]:
#@title Imports
import pandas as pd

In [2]:
#@title File Upload (For Google Colab only)
#@markdown This notebook was created and completed in Google Colab and uses file upload to import the CSV file.
#@markdown
#@markdown Please do not run this cell on Anaconda / Jupyter Notebook.

from google.colab import files
uploaded = files.upload()

Saving data2.csv to data2.csv


In [3]:
#@title Read Data / View Head
df = pd.read_csv("data2.csv", names = ["Type", "Agent IP", "Input Port", "Output Port", "Source MAC", "Destination MAC", "Ethernet Type", "Input VLAN", "Output VLAN", "Source IP", "Destination IP", "IP Protocol", 
                                              "IP Type of Service", "IP Time-To-Live", "Source Port", "Destination Port", "TCP Flags", "Packet Size", "IP Packet Size", 
                                                                                 "Sampling Rate", "TEST"])
df = df.iloc[:, :-1]
print(df.head())

   Type     Agent IP  Input Port  ...  Packet Size IP Packet Size Sampling Rate
0  FLOW  aa.aa.aa.aa         137  ...         1518           1500          2048
1  FLOW  aa.aa.aa.aa         129  ...           74             52          2048
2  FLOW  aa.aa.aa.aa         137  ...         1518           1500          2048
3  FLOW  aa.aa.aa.aa         129  ...          116             94          2048
4  FLOW  aa.aa.aa.aa         130  ...           70             52          2048

[5 rows x 20 columns]


In [4]:
#@title 4A: Top 5 Talkers
df_source_ip_count = df["Source IP"].value_counts().rename_axis("IP Adddress").reset_index(name="Packets")
df_source_ip_count.index += 1
print(df_source_ip_count.head(5))

     IP Adddress  Packets
1   193.62.192.8     3041
2  155.69.160.32     2975
3  130.14.250.11     2604
4  14.139.196.58     2452
5  140.112.8.139     2056


In [5]:
#@title 4A: Top 5 Listeners
df_destination_ip_count = df["Destination IP"].value_counts().rename_axis('IP Adddress').reset_index(name='Packets')
df_destination_ip_count.index += 1
print(df_destination_ip_count.head(5))

       IP Adddress  Packets
1   103.37.198.100     3841
2   137.132.228.15     3715
3   202.21.159.244     2446
4  192.101.107.153     2368
5     103.21.126.2     2056


In [6]:
#@title 4B: Transport Protocol Proportion
total_packet_count = df["IP Protocol"].count()
df_protocol_packets = df["IP Protocol"].value_counts().rename_axis("Header").reset_index(name = "Packets")
df_protocol_packets.index += 1
print(df_protocol_packets)

df_total = pd.DataFrame({"Header":["-"], "Packets":[total_packet_count]})
df_tcp = df.loc[df['IP Protocol'] == 6]["IP Protocol"].value_counts().rename_axis("Header").reset_index(name='Packets')
df_udp = df.loc[df['IP Protocol'] == 17]["IP Protocol"].value_counts().rename_axis("Header").reset_index(name='Packets')
df_tcp_udp = pd.concat([df_total, df_tcp, df_udp]).reset_index(drop = True)
df_tcp_udp.insert(1, "Protocol", ["All", "TCP", "UDP"])
print()

packet_count = df_tcp_udp["Packets"].values
tcp_proportion = packet_count[1] / total_packet_count * 100
udp_proportion = packet_count[2] / total_packet_count * 100

df_tcp_udp.insert(len(df_tcp_udp), "Proportion (%)", ["100", tcp_proportion, udp_proportion])
df_tcp_udp.index += 1
print(df_tcp_udp.head())

    Header  Packets
1        6    56064
2       17     9462
3       50     1698
4        0     1261
5       47      657
6       41      104
7        1       74
8      381       45
9       58        4
10     103        1

  Header Protocol  Packets Proportion (%)
1      -      All    69370            100
2      6      TCP    56064        80.8188
3     17      UDP     9462        13.6399


In [7]:
#@title 4C: Top 5 Application Protocol
df_app_protocol_count = df["Destination Port"].value_counts().rename_axis('Port').reset_index(name='Packets')
df_app_protocol_count.index += 1
print(df_app_protocol_count.head(5))

    Port  Packets
1    443    13423
2     80     2647
3  52866     2068
4  45512     1356
5  56152     1341


In [12]:
#@title 4D: Traffic Intensity (Not Completed)
total_packet_count = df.shape[0]
print("Total Packet Count:", total_packet_count)

total_packet_size = df["IP Packet Size"].sum()
print("Total Traffic:", total_packet_size / 1000 / 1000, "MB")

Total Packet Count: 69370
Total Traffic: 64.777822 MB


In [11]:
#@title [Optional] Top 5 Communication Pair
#test1 = pd.DataFrame({"Source IP":[1, 2, 3, 4, 1, 1, 1, 2, 5], "Destination IP":[2, 1, 4, 3, 2, 2, 2, 1, 6], "BLA":[1,2,3,4,5,6,7,8,9]})
#test2 = test1.groupby(["Source IP", "Destination IP"]).size().reset_index(name = "Count")
#print(test2)
test2 = df.groupby(["Source IP", "Destination IP"]).size().reset_index(name = "Count")

frame_size = test2.shape[0]
source_ip_array = []
destination_ip_array = []
count_array = []
drop_index_array = []
print("Running... takes awhile...")
for index in range(frame_size):
    
    if (index in drop_index_array):
        continue
    
    source_ip_1 = test2.iloc[index]["Source IP"]
    destination_ip_1 = test2.iloc[index]["Destination IP"]
    count_1 = test2.iloc[index]["Count"]
    index2 = test2.index[(test2["Destination IP"] == source_ip_1) & (test2["Source IP"] == destination_ip_1) & (~test2.index.isin(drop_index_array))]
    #print(index2)
    if index2.empty:
        source_ip_array.append(source_ip_1)
        destination_ip_array.append(destination_ip_1)
        count_array.append(count_1)
    else:
        drop_index_array.append(index2[0])
        count_2 = test2.iloc[index2[0]]["Count"]
        source_ip_array.append(source_ip_1)
        destination_ip_array.append(destination_ip_1)
        count_array.append(count_1 + count_2)
    
    #test2.loc[test2['Source IP'] == 6]["IP Protocol"]
    
test3 = pd.DataFrame({"IP Address 1":source_ip_array, "IP Address 2":destination_ip_array, "Count":count_array})
print(test3.sort_values(by = ["Count"], ascending=False).head(5))

Running... takes awhile...
        IP Address 1     IP Address 2  Count
1237  137.132.228.15     193.62.192.8   4951
212   103.37.198.100    130.14.250.11   2842
1426   14.139.196.58  192.101.107.153   2368
1553   140.112.8.139     103.21.126.2   2056
1585   140.90.101.61     167.205.52.8   1752
